In [2]:
import os
import time
import tensorflow as tf
import numpy as np


In [8]:

LABEL_DIMENSIONS = 10
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.fashion_mnist.load_data()
TRAIN_SIZE = len(train_images)
TEST_SIZE = len(test_images)


In [6]:
# re-shaping the input
train_images = np.asanyarray(train_images, dtype=np.float32) / 255
train_images = train_images.reshape((TRAIN_SIZE, 28, 28, 1))
test_images = np.asanyarray(test_images, dtype=np.float32) / 255
test_images = test_images.reshape((TEST_SIZE, 28, 28, 1))


In [9]:
# one hot elements
train_labels = tf.keras.utils.to_categorical(train_labels, LABEL_DIMENSIONS)
test_labels = tf.keras.utils.to_categorical(test_labels, LABEL_DIMENSIONS)

train_labels = train_labels.astype(np.float32)
test_labels = test_labels.astype(np.float32)
print(train_labels.shape)
print(test_labels.shape)


(60000, 10)
(10000, 10)


In [24]:
# How many categories we are predicting from (0-9)
LABEL_DIMENSIONS = 10


(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.fashion_mnist.load_data()
TRAINING_SIZE = len(train_images)
TEST_SIZE = len(test_images)

train_images = np.asarray(train_images, dtype=np.float32) / 255

# Convert the train images and add channels
train_images = train_images.reshape((TRAINING_SIZE, 28, 28, 1))

test_images = np.asarray(test_images, dtype=np.float32) / 255
# Convert the train images and add channels
test_images = test_images.reshape((TEST_SIZE, 28, 28, 1))

train_labels  = tf.keras.utils.to_categorical(train_labels, LABEL_DIMENSIONS)
test_labels = tf.keras.utils.to_categorical(test_labels, LABEL_DIMENSIONS)

# Cast the labels to float
train_labels = train_labels.astype(np.float32)
test_labels = test_labels.astype(np.float32)
print (train_labels.shape)
print (test_labels.shape)


(60000, 10)
(10000, 10)


In [15]:
# CNN model - using functional API
inputs = tf.keras.Input(shape=(28, 28, 1))
x = tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu')(inputs)
x = tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=2)(x)
x = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)

predictions = tf.keras.layers.Dense(LABEL_DIMENSIONS, activation='softmax')(x)
model = tf.keras.Model(inputs=inputs, outputs= predictions)
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_9 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 13, 13, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_10 (Conv2D)          (None, 11, 11, 64)        18496     
                                                                 
 flatten_4 (Flatten)         (None, 7744)              0         
                                                                 
 dense_9 (Dense)             (None, 64)                495680    
                                                             

In [17]:
# compiling the model
optimizer = tf.keras.optimizers.SGD()
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

## Defining  The Strategy

In [18]:
strategy = tf.distribute.MirroredStrategy()
config = tf.estimator.RunConfig(train_distribute=strategy)

Instructions for updating:
Use tf.keras instead.


In [20]:
# conveting keras to estimator
estimator = tf.keras.estimator.model_to_estimator(model, config=config)

/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:452: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn(
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.


In [21]:
# we use this function for estimators
def input_fn(images, labels, epochs, batch_size):
  dataset = tf.data.Dataset.from_tensor_slices((images, labels))
  SHUFFLE_SIZE = 5000
  dataset = dataset.shuffle(SHUFFLE_SIZE).repeat(epochs).batch(batch_size)
  dataset = dataset.prefetch(None)
  return dataset

In [25]:
# train based on the function
# we apply the function as the time of traning

BATCH_SIZE = 512
EPOCHS = 50

estimator_train_result = estimator.train(input_fn=lambda:input_fn(train_images,
                                         train_labels,
                                         epochs=EPOCHS,
                                         batch_size=BATCH_SIZE))
print(estimator_train_result)


In [26]:
estimator.evaluate(lambda:input_fn(test_images, test_labels, epochs=1, batch_size=BATCH_SIZE))

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training_v1.py:2335: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates
Instructions for updating:
Use tf.keras instead.


{'accuracy': 0.8504, 'loss': 0.40788525, 'global_step': 5860}